## Краткая демонстрация к тестовому заданию

Для работы с базой данных нам понадобиться:
1. Сама база данных, поднимем бесплатную PostgreSQL на [filess.io](https://filess.io/).
2. Фреймворк [SQLAlchemy](https://www.sqlalchemy.org/).
3. Немного самодельных классов и функций.

Определим:
* `config.py` - параметры для установки соединения;
* `database.py` - параметры подключения к БД;
* `models.py` - классы и объекты с которыми мы будем работать (структуры таблиц);
* `core.py` - функции для работы в core (прямом) режиме;
* `orm.py` - функции для работы в ORM (объектном) режиме;
* `main.py` - непосредственная работа с базой данных;

Создаём и наполняем данными таблицы interns, grades и tools

In [10]:
from orm import SyncOrm

SyncOrm.create_tables()
SyncOrm.insert_interns()
SyncOrm.insert_grades()
SyncOrm.insert_tools()

Посмотрим, как составлять базовые запросы

Обратимся к базе данных и выведем всю таблицу **interns** экземплярами класса `InternsOrm`, который имеет следующие атрибуты:
```python
class InternsOrm(Base):
    __tablename__ = "interns"
    __table_args__ = {"schema": "cloveri"}
    id:         Mapped[str] = mapped_column(primary_key=True)
    email:      Mapped[str]
    name:       Mapped[str]
    role:       Mapped[str]
    internship: Mapped[str]
    start_date: Mapped[datetime.datetime]
    end_date:   Mapped[datetime.datetime]
    notes:      Mapped[str | None]
```

In [12]:
from sqlalchemy import select
from database import session_factory
from models import InternsOrm

with session_factory() as session:
    stmt = select(InternsOrm)
    result = session.execute(stmt)
    interns = result.scalars().all()
    for model in interns:
        print(model.id, model.username, model.internship)

f1 Miguel Aubakirov Кабинет Кловери
f2 Miguel Aubakirov MAIBO
f3 Nombre Apellido Кабинет Кловери
f4 Jose Rodrigues MAIBO
f5 Soledad Lorna Платформа для ВУЗов


Простой запрос с фильтрацией, посмотрим сколько записей в таблице interns есть для пользователя с адресом электронной почты `work.aubakirov@gmail.com`

In [13]:
from sqlalchemy import select
from database import session_factory
from models import InternsOrm

intern_email = 'work.aubakirov@gmail.com'

with session_factory() as session:
    query = (
        select(InternsOrm)
        .filter(InternsOrm.email.contains(intern_email))
    )
    # выведем на экран сырой запрос, который отправится в базу
    print(f"{query.compile(compile_kwargs={"literal_binds": True})}\n")
    result = session.execute(query)
    interns = result.scalars().all()
    for model in interns:
        print(model.id, model.username, model.internship)

SELECT cloveri.interns.id, cloveri.interns.email, cloveri.interns.username, cloveri.interns.role, cloveri.interns.internship, cloveri.interns.start_date, cloveri.interns.end_date, cloveri.interns.notes 
FROM cloveri.interns 
WHERE (cloveri.interns.email LIKE '%' || 'work.aubakirov@gmail.com' || '%')

f1 Miguel Aubakirov Кабинет Кловери
f2 Miguel Aubakirov MAIBO


Посмотрим количество стажеров в каждом проекте

In [14]:
from sqlalchemy import select, func
from database import session_factory
from models import InternsOrm

with session_factory() as session:
    query = (
        select(InternsOrm.internship, func.count(InternsOrm.internship))
        .group_by(InternsOrm.internship)
        .order_by(InternsOrm.internship)
    )
    # выведем на экран сырой запрос, который отправится в базу
    print(f"{query.compile(compile_kwargs={"literal_binds": True})}\n")
    result = session.execute(query)
    interns = result.all()
    for model in interns:
        print(model)

SELECT cloveri.interns.internship, count(cloveri.interns.internship) AS count_1 
FROM cloveri.interns GROUP BY cloveri.interns.internship ORDER BY cloveri.interns.internship

('MAIBO', 2)
('Кабинет Кловери', 2)
('Платформа для ВУЗов', 1)


In [15]:
from sqlalchemy import select, func
from database import session_factory
from models import ToolsOrm

with session_factory() as session:
    stmt = select(ToolsOrm)
    print(f"{stmt.compile(compile_kwargs={"literal_binds": True})}\n")
    #result = session.execute(stmt)
    #interns = result.scalars().all()
    #for model in interns:
    #    print(model)

SELECT cloveri.tools.id, cloveri.tools.role, cloveri.tools.internship, cloveri.tools.pycaret, cloveri.tools.random_forest, cloveri.tools."anomaly_detеction", cloveri.tools."gradient_Boosting", cloveri.tools.cluster_analysis, cloveri.tools.k_fold, cloveri.tools.optuna, cloveri.tools.shap, cloveri.tools.tsfresh, cloveri.tools.sklearn_feature_selection, cloveri.tools.imblearn, cloveri.tools.streamlit, cloveri.tools.selenium, cloveri.tools.allure, cloveri.tools.nose, cloveri.tools."simpleTest", cloveri.tools.docker, cloveri.tools.jira, cloveri.tools.adobe_xd, cloveri.tools.figma, cloveri.tools.mockplus, cloveri.tools.yandex_metrika, cloveri.tools.google_analytics, cloveri.tools.sketch, cloveri.tools.fontjoy, cloveri.tools.sqlalchemy, cloveri.tools.postgresql, cloveri.tools.git, cloveri.tools.django 
FROM cloveri.tools

